<a href="https://colab.research.google.com/github/SevillaBK/Tensorflow2.0/blob/master/Chapter_6_%EC%BB%A8%EB%B3%BC%EB%A3%A8%EC%85%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%84%B1%EB%8A%A5_%ED%96%A5%EC%83%81%EC%8B%9C%ED%82%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

컨볼루션 신경망의 성능을 높이기 위해서 사용되는 방법에 대해 알아보겠습니다.<br/>
대표적으로 쓰이는 두 방법은 **레이어 많이 쌓기**와 **이미지 보강(Image Augmentation)**입니다.

# 1. 더 많은 레이어 쌓기

딥러닝에서 네트워크의 구조를 깊게 쌓는 것이 가능해진 후 컨볼루션 신경망에서는 컨볼루션 레이어가<br/>
중첩된 더 깊은 구조가 계속해서 나타났고, 그 때마다 퍼포먼스가 개선되었습니다.

아래에서는 깊은 컨볼루션 레이어를 쌓는 방법 중 하나인 **VGGNet**의 스타일의 신경망으로<br/>
Fashion MNIST 데이터셋에 적용해보겠습니다.

In [3]:
# 텐서플로 불러오기
import tensorflow as tf

# Fashion MNIST 데이터셋 불러오기
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

# 데이터 스케일링
train_X = train_X / 255.0
test_X = test_X / 255.0

# reshape 이전
print(train_X.shape, test_X.shape)

train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

# reshape 이후
print(train_X.shape, test_X.shape)

(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [4]:
# VGGNet 스타일의 콘볼루션 신경망 모델
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3, 3), filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 256)        

VGGNet은 여러 구조로 실험했는데 그 중 19개의 레이어가 겹쳐진 VGG-19가 제일 깊은 구조입니다.<br/>
VGG-19는 초반에 컨볼루션 레이어를 2개 겹친 뒤 풀링레이어 1개를 사용하는 패턴을 2차례,<br/>
그 후 컨볼루션 레이어를 4개 겹친 뒤 풀링 레이어를 1개 사용하는 패턴을 3차례 반복합니다.

![alt text](https://github.com/injo-image/image/blob/master/tensorflow/chapter6/VGG-19.png?raw=true)

In [0]:
# VGGNet 스타일의 컨볼루션 신경망 모델 학습
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

plt.